In [50]:
import pandas as pd
import sqlite3
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [8]:
con=sqlite3.connect('amazon_reviews/database.sqlite')

In [23]:
data=pd.read_sql_query('''select * from Reviews where score !=3 limit 10000''',con)
data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [24]:
def partition(x):
    if x<3:
        return 0
    return 1
positivenegative=data['Score'].map(partition)
data['Score']=positivenegative
data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,1,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,1,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,1,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,1,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,1,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [27]:
data=data.drop_duplicates(subset={'UserId','Score','Time','Text'},keep='first',inplace=False)

In [28]:
data.shape

(9564, 10)

In [34]:
from nltk.corpus import stopwords
from gensim.models import Word2Vec
stopword=stopwords.words('english')
stop=[]
for i in stopword:
    if i not in ['not','no']:
        stop.append(i)


In [45]:
from tqdm import tqdm
from bs4 import BeautifulSoup

preprocessed_review=[]
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

for sentence in tqdm((data['Text'])):
    sentence=re.sub(r"http\s+","",sentence)
    sentence=re.sub('[^A-Za-z]',' ',sentence) #Removing special characters ^ means negate
    sentence=BeautifulSoup(sentence,'lxml').get_text()
    sentence=re.sub("\S*\d\S*","",sentence) #Removing Alphannumeric characters
    sentence=decontracted(sentence)
    sentence=' '.join(e.lower() for e in sentence.split() if e.lower not in stop)
    preprocessed_review.append(sentence.strip())
    
for sentence in tqdm((data['Summary'])):
    sentence=re.sub(r"http\s+","",sentence)
    sentence=re.sub('[^A-Za-z]',' ',sentence) #Removing special characters ^ means negate
    sentence=BeautifulSoup(sentence,'lxml').get_text()
    sentence=re.sub("\S*\d\S*","",sentence) #Removing Alphannumeric characters
    sentence=decontracted(sentence)
    sentence=' '.join(e.lower() for e in sentence.split() if e.lower not in stop)
    preprocessed_review.append(sentence.strip())
    

100%|████████████████████████████████████████████████████████████████████████████| 9564/9564 [00:05<00:00, 1868.29it/s]


In [70]:
preprocessed_review

['i have bought several of the vitality canned dog food products and have found them all to be of good quality the product looks more like a stew than a processed meat and it smells better my labrador is finicky and she appreciates this product better than most',
 'product arrived labeled as jumbo salted peanuts the peanuts were actually small sized unsalted not sure if this was an error or if the vendor intended to represent the product as jumbo',
 'this is a confection that has been around a few centuries it is a light pillowy citrus gelatin with nuts in this case filberts and it is cut into tiny squares and then liberally coated with powdered sugar and it is a tiny mouthful of heaven not too chewy and very flavorful i highly recommend this yummy treat if you are familiar with the story of c s lewis the lion the witch and the wardrobe this is the treat that seduces edmund into selling out his brother and sisters to the witch',
 'if you are looking for the secret ingredient in robitus

<h1>Featurization</h1>

<h2>Bag of words</h2>

In [56]:
count_vect=CountVectorizer()
count_vect.fit(preprocessed_review)
print(count_vect.get_feature_names()[:10])
final_count=count_vect.transform(preprocessed_review)
final_count

['aa', 'aaaa', 'aaaarrrrrgggghhhhh', 'aacute', 'aahhhs', 'ab', 'aback', 'abandon', 'abates', 'abberline']


<19128x18792 sparse matrix of type '<class 'numpy.int64'>'
	with 525126 stored elements in Compressed Sparse Row format>

## N-Grams and Bi-Grams

In [59]:
count_vect=CountVectorizer(ngram_range=(1,2),min_df=10,max_features=5000)
final_bigram_counts=count_vect.fit_transform(preprocessed_review)
final_bigram_counts.get_shape()

(19128, 5000)

## TF-IDF 

In [61]:
tfidf_vect=TfidfVectorizer(ngram_range=(1,2),max_df=10,max_features=5000)
final_tfidf_count=tfidf_vect.fit_transform(preprocessed_review)
final_tfidf_count.get_shape()

(19128, 5000)

## Word2Vec

In [73]:
import gensim
list_of_sentence=[]
for sentence in preprocessed_review:
    list_of_sentence.append(sentence.split())
list_of_sentence

[['i',
  'have',
  'bought',
  'several',
  'of',
  'the',
  'vitality',
  'canned',
  'dog',
  'food',
  'products',
  'and',
  'have',
  'found',
  'them',
  'all',
  'to',
  'be',
  'of',
  'good',
  'quality',
  'the',
  'product',
  'looks',
  'more',
  'like',
  'a',
  'stew',
  'than',
  'a',
  'processed',
  'meat',
  'and',
  'it',
  'smells',
  'better',
  'my',
  'labrador',
  'is',
  'finicky',
  'and',
  'she',
  'appreciates',
  'this',
  'product',
  'better',
  'than',
  'most'],
 ['product',
  'arrived',
  'labeled',
  'as',
  'jumbo',
  'salted',
  'peanuts',
  'the',
  'peanuts',
  'were',
  'actually',
  'small',
  'sized',
  'unsalted',
  'not',
  'sure',
  'if',
  'this',
  'was',
  'an',
  'error',
  'or',
  'if',
  'the',
  'vendor',
  'intended',
  'to',
  'represent',
  'the',
  'product',
  'as',
  'jumbo'],
 ['this',
  'is',
  'a',
  'confection',
  'that',
  'has',
  'been',
  'around',
  'a',
  'few',
  'centuries',
  'it',
  'is',
  'a',
  'light',
  'pil

In [87]:
#Creating our own Word2vec
w2v_model=gensim.models.Word2Vec(list_of_sentence,min_count=10,size=50,workers=4)

In [92]:
w2v_model.wv.most_similar('tasty')

[('yummy', 0.9114243388175964),
 ('satisfying', 0.9106889367103577),
 ('delicious', 0.8386292457580566),
 ('soft', 0.8262532949447632),
 ('light', 0.8192130327224731),
 ('thin', 0.8146151304244995),
 ('flavorful', 0.8097797632217407),
 ('salty', 0.7883036136627197),
 ('crunchy', 0.7882554531097412),
 ('thick', 0.7855238914489746)]

In [93]:
w2v_model.wv.similarity('tasty','delicious')

0.8386292643321913

In [94]:
w2v_model.wv.similarity('tasty','nice')

0.6356999785954047